In [346]:
import pandas as pd
import os
import sklearn
import math
import numpy as np
import matplotlib
from datetime import date
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import csv
from urllib.request import urlopen
from datetime import datetime
from dataprep.eda import plot
from dataprep.eda import plot_correlation
from dataprep.eda import plot_missing

In [347]:
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
def get_continent(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return (cn_continent)

def get_code(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return (cn_a2_code)

def Normalize_cols (df, cols):
    for i in cols:
        normalize(df,i) 
def normalize(df, col):
    max_value = df[col].max()
    min_value = df[col].min()
    df[col] = (df[col] - min_value) / (max_value - min_value)

In [348]:
def graphi(df, string):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df["date"], y=df["rollretail"], name="Retail Index"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df["date"], y=df["RollCases"], name="New Confirmed Cases "+string+" change(Avg 5 days)"),
        secondary_y=True,
    )
    fig.add_trace(
        go.Scatter(x=df["date"], y=df["Rollrecover"], name="New Recovered Cases "+string+" change(Avg 5 days)"),
        secondary_y=True,
    )
    fig.add_trace(
        go.Scatter(x=df["date"], y=df["Government Response Stringency Index ((0 to 100, 100 = strictest))"], name="Government Response Index"),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df["date"], y=df["Close"], name=string+" Index"),
        secondary_y=False,
    )

    # Add figure title
    fig.update_layout(
        title_text=string+" Deep Dive",
        plot_bgcolor='rgb(14,28,68)',
        legend=dict(
            x=1.2,
            y=0.5)
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Date",showgrid=False)

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Normalized Indexs Values", secondary_y=False,showgrid=False)
    fig.update_yaxes(title_text="<b>COVID19 Cases</b> Real Numbers", secondary_y=True,showgrid=False)

    fig.show(plot_bgcolor='rgb(14,28,68)')

In [349]:
## Uploading covid data
url = "https://raw.githubusercontent.com/datasets/covid-19/master/data/countries-aggregated.csv"
response = urlopen(url).read().decode('ascii','ignore')
df_covid = csv.reader(response)
df_covid = pd.read_csv(url) 
#df_covid = pd.DataFrame(df_covid)
df_covid['Date'] = df_covid['Date'].astype('datetime64[ns]') 
df_covid["CountryCode"] = 0
df_covid["Continent"] = 0
i=0
while i < len(df_covid):
    x = df_covid["Country"].iloc[i]
    df_covid["Continent"].iloc[i] = get_continent(x)
    df_covid["CountryCode"].iloc[i] = get_code(x)
    i=i+1
df_covid.loc[df_covid['Country'] == "US", ['CountryCode']] = 'USA'
df_covid.loc[df_covid['Country'] == "Korea, South", ['Country']] = 'South Korea'
df_covid.loc[df_covid['Country'] == "South Korea", ['CountryCode']] = 'KOR'

C:\Users\shtay\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [350]:
## Uploading Gov data
datafile = 'covidstr.csv'
#Read csv file using pandas
df_str = pd.read_csv(datafile)
df_str['Date'] = df_str['Date'].astype('datetime64[ns]') 
normalize(df_str, "Government Response Stringency Index ((0 to 100, 100 = strictest))")
df_str.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20264 entries, 0 to 20263
Data columns (total 4 columns):
 #   Column                                                              Non-Null Count  Dtype         
---  ------                                                              --------------  -----         
 0   Date                                                                20264 non-null  datetime64[ns]
 1   Entity                                                              20264 non-null  object        
 2   Code                                                                20264 non-null  object        
 3   Government Response Stringency Index ((0 to 100, 100 = strictest))  20264 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 633.4+ KB


In [351]:
## Uploading google data
datafile = 'Global_Mobility_Report.csv'
#Read csv file using pandas
df_gomob = pd.read_csv(datafile)
if df_gomob["date"].dtype == np.object:
    try:
        df_gomob['date'] = df_gomob['date'].astype('datetime64[ns]') 
    except ValueError:
        pass
df_gomob.info()
df_gomob["CountryCode"] = 0
df_gomob["Continent"] = 0
i=0
while i < len(df_covid):
    x = df_gomob["country_region"].iloc[i]
    df_gomob["Continent"].iloc[i] = get_continent(x)
    df_gomob["CountryCode"].iloc[i] = get_code(x)
    i=i+1
df_gomob.loc[df_gomob['country_region'] == "United States", ['CountryCode']] = 'USA'
df_gomob.loc[df_gomob['country_region'] == "United States", ['country_region_code']] = 'USA'
df_gomob.loc[df_gomob['country_region'] == "South Korea", ['country_region_code']] = 'KOR'
df_gomob.loc[df_gomob['country_region'] == "South Korea", ['CountryCode']] = 'KOR'

C:\Users\shtay\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning:

Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335463 entries, 0 to 335462
Data columns (total 11 columns):
 #   Column                                              Non-Null Count   Dtype         
---  ------                                              --------------   -----         
 0   country_region_code                                 335378 non-null  object        
 1   country_region                                      335463 non-null  object        
 2   sub_region_1                                        324243 non-null  object        
 3   sub_region_2                                        224707 non-null  object        
 4   date                                                335463 non-null  datetime64[ns]
 5   retail_and_recreation_percent_change_from_baseline  271455 non-null  float64       
 6   grocery_and_pharmacy_percent_change_from_baseline   262967 non-null  float64       
 7   parks_percent_change_from_baseline                  145344 non-null  float64       

In [352]:
new_df = pd.merge(df_gomob, df_covid, left_on=['date','country_region_code'], right_on = ['Date','CountryCode'], how="outer")
new_df = new_df.drop(['Date','sub_region_1','sub_region_2','country_region_code'], axis = 1) 
new_df.loc[new_df['country_region'] == "United States", ['CountryCode_x']] = 'USA'
new_df.loc[new_df['country_region'] == "United States", ['Country']] = 'United States of America'

In [353]:
Moti = new_df.groupby(["date","Country"], as_index=False).mean()
Moti["CountryCode"] = 0
Moti["Continent"] = 0
i=0
while i < len(Moti):
    x = Moti["Country"].iloc[i]
    Moti["Continent"].iloc[i] = get_continent(x)
    Moti["CountryCode"].iloc[i] = get_code(x)
    i=i+1

In [354]:
country = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
country = country.drop(['GDP (BILLIONS)'], axis = 1) 
country.loc[country['COUNTRY'] == "United States", ['COUNTRY']] = 'United States of America'
country.loc[country['COUNTRY'] == "Korea, South", ['COUNTRY']] = 'South Korea'
country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222 entries, 0 to 221
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   COUNTRY  222 non-null    object
 1   CODE     222 non-null    object
dtypes: object(2)
memory usage: 3.6+ KB


In [355]:
Moti = pd.merge(Moti, country, left_on='Country', right_on='COUNTRY', how="left")
Moti = pd.merge(Moti, df_str, left_on=['date','CODE'], right_on=['Date','Code'], how="left")

In [356]:
Moti.loc[Moti['Country'] == "Korea, South", ['Country']] = 'South Korea'

In [357]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')

fig = go.Figure(data=go.Choropleth(
    locations = Moti['CODE'],
    z = Moti['retail_and_recreation_percent_change_from_baseline'],
    text = Moti["Country"],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '',
    colorbar_title = 'Retail & Recreation index',
))

fig.update_layout(
    title_text='Retail & Recreation Index',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Moti Berliner',
        showarrow = False
    )]
)

fig.show()

In [358]:
Moti.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10540 entries, 0 to 10539
Data columns (total 19 columns):
 #   Column                                                              Non-Null Count  Dtype         
---  ------                                                              --------------  -----         
 0   date                                                                10540 non-null  datetime64[ns]
 1   Country                                                             10540 non-null  object        
 2   retail_and_recreation_percent_change_from_baseline                  10472 non-null  float64       
 3   grocery_and_pharmacy_percent_change_from_baseline                   10440 non-null  float64       
 4   parks_percent_change_from_baseline                                  10408 non-null  float64       
 5   transit_stations_percent_change_from_baseline                       10449 non-null  float64       
 6   workplaces_percent_change_from_baseline               

In [359]:
Moti = Moti.drop(['residential_percent_change_from_baseline','workplaces_percent_change_from_baseline','transit_stations_percent_change_from_baseline','parks_percent_change_from_baseline','Date','Entity','Continent'], axis = 1) 

In [360]:
def addcountry(Countryx):
    Countryx["confirmeddelta"] = (Countryx["Confirmed"].pct_change())*100
    Countryx["rollconfirmed"] = Countryx.confirmeddelta.rolling(window=5).mean()
    Countryx["NewCases"] = Countryx["Confirmed"].diff(periods=1)
    Countryx["Newdeaths"] = Countryx["Deaths"].diff(periods=1)
    Countryx["Newrecovered"] = Countryx["Recovered"].diff(periods=1)
    Countryx["rollretail"] = Countryx.retail_and_recreation_percent_change_from_baseline.rolling(window=5).mean()
    Countryx["rollgrocery"] = Countryx.grocery_and_pharmacy_percent_change_from_baseline.rolling(window=5).mean()
    Countryx["RollCases"] = Countryx.NewCases.rolling(window=5).mean()
    Countryx["Rolldeath"] = Countryx.Newdeaths.rolling(window=5).mean()
    Countryx["Rollrecover"] = Countryx.Newrecovered.rolling(window=5).mean()   
    
def addmore(coun, cols):
    addcountry(coun)
    Normalize_cols(coun, cols)
    
def normalizecases(coun):
    normalize(Italy,'NewCases')
    normalize(Italy,'Newdeaths')
    normalize(Italy,'Newrecovered')
    Italy["RollCases"] = Italy.NewCases.rolling(window=5).mean()
    Italy["Rolldeath"] = Italy.Newdeaths.rolling(window=5).mean()
    Italy["Rollrecover"] = Italy.Newrecovered.rolling(window=5).mean()
    
def interpolate_missing(df, col):
    df[col] = df[col].interpolate(method='linear', limit=5).fillna(method='ffill').fillna(method='bfill')


In [361]:
Italy = Moti[Moti["Country"]=="Italy"]
France = Moti[Moti["Country"]=="France"]
Sweden = Moti[Moti["Country"]=="Sweden"]
United_States = Moti[Moti["Country"]=="United States of America"]
Germany = Moti[Moti["Country"]=="Germany"]
UK = Moti[Moti["Country"]=="United Kingdom"]
SouthK = Moti[Moti["Country"]=="South Korea"]
Israel = Moti[Moti["Country"]=="Israel"]
Singapore = Moti[Moti["Country"]=="Singapore"]

In [362]:
colus= ['retail_and_recreation_percent_change_from_baseline','rollretail','rollgrocery',
       'grocery_and_pharmacy_percent_change_from_baseline','Government Response Stringency Index ((0 to 100, 100 = strictest))']

In [363]:
addmore(Italy, colus)
addmore(France, colus)
addmore(Sweden, colus)
addmore(United_States, colus)
addmore(Germany, colus)
addmore(UK, colus)
addmore(SouthK, colus)
addmore(Israel, colus)
addmore(Singapore, colus)

C:\Users\shtay\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\shtay\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\shtay\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [364]:
#Read German Dax
datafile = 'GermanDax.csv'
germandax = pd.read_csv(datafile)
germandax['Date'] = germandax['Date'].astype('datetime64[ns]')
germandax["Close"].fillna(method='bfill', inplace=True)
normalize(germandax, "Close")

#Read british FTSE
datafile = 'BritishFTSE.csv'
British = pd.read_csv(datafile)
British['Date'] = British['Date'].astype('datetime64[ns]')
British["Close"].fillna(method='bfill', inplace=True)
normalize(British, "Close")

#Read S&P USA
datafile = 'SPUS.csv'
SandP = pd.read_csv(datafile)
SandP['Date'] = SandP['Date'].astype('datetime64[ns]')
SandP["Close"].fillna(method='bfill', inplace=True)
normalize(SandP, "Close")

#Read Italy MIB
datafile = 'ItalyMIB.csv'
ItalyMIB = pd.read_csv(datafile)
ItalyMIB['Date'] = ItalyMIB['Date'].astype('datetime64[ns]')
ItalyMIB["Close"].fillna(method='bfill', inplace=True)
normalize(ItalyMIB, "Close")

#Read Singapore1
datafile = 'STISingapore.csv'
Singapore1 = pd.read_csv(datafile)
Singapore1['Date'] = Singapore1['Date'].astype('datetime64[ns]')
Singapore1["Close"].fillna(method='bfill', inplace=True)
normalize(Singapore1, "Close")

#Read Israel1
datafile = 'TA100.csv'
Israel1 = pd.read_csv(datafile)
Israel1['Date'] = Israel1['Date'].astype('datetime64[ns]')
Israel1["Close"].fillna(method='bfill', inplace=True)
normalize(Israel1, "Close")

#Read Sweden1
datafile = 'SweOMX.csv'
Sweden1 = pd.read_csv(datafile)
Sweden1['Date'] = Sweden1['Date'].astype('datetime64[ns]')
Sweden1["Close"].fillna(method='bfill', inplace=True)

normalize(Sweden1, "Close")

#Read SouthK
datafile = 'KSKorea.csv'
SouthK1 = pd.read_csv(datafile)
SouthK1['Date'] = SouthK1['Date'].astype('datetime64[ns]')
normalize(SouthK1, "Close")
SouthK.fillna(method='bfill', inplace=True)
#Italy = pd.merge(Italy, df_covid, left_on=['date'], right_on = ['Date'], how="outer")
United_States = pd.merge(United_States, SandP, left_on=['date'], right_on = ['Date'], how="outer")
UK = pd.merge(UK, British, left_on=['date'], right_on = ['Date'], how="outer")
Germany = pd.merge(Germany, germandax, left_on=['date'], right_on = ['Date'], how="outer")
Italy = pd.merge(Italy, ItalyMIB, left_on=['date'], right_on = ['Date'], how="outer")
Singapore = pd.merge(Singapore, Singapore1, left_on=['date'], right_on = ['Date'], how="outer")
Sweden = pd.merge(Sweden, Sweden1, left_on=['date'], right_on = ['Date'], how="outer")
Israel = pd.merge(Israel, Israel1, left_on=['date'], right_on = ['Date'], how="outer")
SouthK = pd.merge(SouthK, SouthK1, left_on=['date'], right_on = ['Date'], how="outer")

C:\Users\shtay\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [365]:
Omri = Italy
Omri = Omri.append(France).append(Sweden).append(United_States).append(Germany).append(UK).append(SouthK).append(Israel)


In [366]:
Singapore["Open"].fillna(method='bfill', inplace=True)
normalize(Singapore, "Open")

In [367]:
Singapore.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328 entries, 0 to 327
Data columns (total 29 columns):
 #   Column                                                              Non-Null Count  Dtype         
---  ------                                                              --------------  -----         
 0   date                                                                85 non-null     datetime64[ns]
 1   Country                                                             85 non-null     object        
 2   retail_and_recreation_percent_change_from_baseline                  85 non-null     float64       
 3   grocery_and_pharmacy_percent_change_from_baseline                   85 non-null     float64       
 4   Confirmed                                                           85 non-null     float64       
 5   Recovered                                                           85 non-null     float64       
 6   Deaths                                                    

In [368]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=Singapore["date"], y=Singapore["rollretail"], name="Retail Index"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=Singapore["date"], y=Singapore["RollCases"], name="New Confirmed Cases Italy change(Avg 5 days)"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=Singapore["date"], y=Singapore["Rollrecover"], name="New Recovered Cases Italy change(Avg 5 days)"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=Singapore["date"], y=Singapore["Government Response Stringency Index ((0 to 100, 100 = strictest))"], name="Government Response Index"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=Singapore["date"], y=Singapore["Open"], name="Singapore FTSE Index"),
    secondary_y=False,
)

# Add figure title
fig.update_layout(
    title_text="Singapore Deep Dive",
    plot_bgcolor='rgb(14,28,68)'
)

# Set x-axis title
fig.update_xaxes(title_text="Date",showgrid=False)

# Set y-axes titles
fig.update_yaxes(title_text="<b>Normalized Values", secondary_y=False,showgrid=False)
fig.update_yaxes(title_text="<b>Retail & Recreation</b> Normalized", secondary_y=True,showgrid=False)

fig.show(plot_bgcolor='rgb(14,28,68)')

In [369]:
Sweden["Close"].fillna(method='bfill', inplace=True)
normalize(Sweden, "Close")

In [370]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=Sweden["date"], y=Sweden["rollretail"], name="Retail Index"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=Sweden["date"], y=Sweden["RollCases"], name="New Confirmed Cases Sweden change(Avg 5 days)"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=Sweden["date"], y=Sweden["Rollrecover"], name="New Recovered Cases Sweden change(Avg 5 days)"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=Sweden["date"], y=Sweden["Government Response Stringency Index ((0 to 100, 100 = strictest))"], name="Government Response Index"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=Sweden["date"], y=Sweden["Close"], name="Sweden FTSE Index"),
    secondary_y=False,
)

# Add figure title
fig.update_layout(
    title_text="Sweden Deep Dive",
    plot_bgcolor='rgb(14,28,68)'
)

# Set x-axis title
fig.update_xaxes(title_text="Date",showgrid=False)

# Set y-axes titles
fig.update_yaxes(title_text="<b>Normalized Values", secondary_y=False,showgrid=False)
fig.update_yaxes(title_text="<b>Retail & Recreation</b> Normalized", secondary_y=True,showgrid=False)

fig.show(plot_bgcolor='rgb(14,28,68)')

In [371]:
United_States["Close"].fillna(method='bfill', inplace=True)
normalize(United_States, "Close")

In [372]:
Italy["Close"].fillna(method='bfill', inplace=True)
normalize(Italy, "Close")

In [373]:
graphi(Italy,"Italy")

In [374]:
graphi(United_States,"United States")

In [375]:
Israel["Close"].fillna(method='bfill', inplace=True)
normalize(Israel, "Close")

In [376]:
graphi(Israel,"Israel")

In [377]:
SouthK["Close"].fillna(method='bfill', inplace=True)
normalize(SouthK, "Close")

In [378]:
graphi(SouthK,"South Korea")

In [379]:
Germany["Close"].fillna(method='bfill', inplace=True)
normalize(Germany, "Close")

In [380]:
graphi(Germany, "Germany")

In [381]:
Sweden["Close"].fillna(method='bfill', inplace=True)
normalize(Sweden, "Close")

In [382]:
graphi(Sweden, "Sweden")

In [ ]:
metest = 